# Ablation tests for group-by component

## Prepare dataset

Loading and preparing dataset as in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb).

In [1]:
DATASET = 'rees_ecommerce'
TECHNICAL_COLUMNS = ['user_id', 'date']
TASKS = ['product_id', 'category1', 'category2', 'category3', 'brand', 'priceCluster']
BATCH_SIZE_TRAIN = 5040
BATCH_SIZE_EVAL = 5040

In [2]:
from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + TECHNICAL_COLUMNS, all_columns))

In [4]:
from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=BATCH_SIZE_TRAIN, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=BATCH_SIZE_EVAL, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

In [5]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

In [6]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

## Prepare evaluation dataset

In [7]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

CPU times: user 7min 23s, sys: 32.2 s, total: 7min 55s
Wall time: 7min 9s


## Model

In [8]:
if DATASET == 'movielens_imdb':
    from model_parameters import movielens_model as model_definition
    from model_parameters import MOVIELENS_EPOCHS as EPOCHS
elif DATASET == 'rees_ecommerce':
    from model_parameters import rees_model as model_definition
    from model_parameters import REES_EPOCHS as EPOCHS

NUMBER_OF_NEGATIVES = 4

In [9]:
import itertools

MODELS, EVAL_MODELS, EMBEDDING_MODELS = {}, {}, {}
for group_by, mask_net, bi_linear_interaction in itertools.product(*([[True, False]] * 3)):
    if mask_net and not group_by:
        continue
    key = group_by, mask_net, bi_linear_interaction
    name = '_'.join(map(lambda x: str(x).lower(), key))
    MODELS[key], EVAL_MODELS[key], EMBEDDING_MODELS[key] = \
        model_definition(user_features, offer_features, inverse_lookups, NUMBER_OF_NEGATIVES, name, *key)

## Train

In [10]:
len(MODELS)

6

In [11]:
for model in MODELS.values():
    model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/8


/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/true_true_true/group_by/moments_2/Reshape_4:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/true_true_true/group_by/moments_2/Reshape_3:0", shape=(None, 6, 100), dtype=float32), dense_shape=Tensor("gradient_tape/true_true_true/group_by/moments_2/Cast:0", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/jupyter/.local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSl

679/679 [==============================] - 651s 817ms/step - loss: 0.4688 - auc: 0.6916 - val_loss: 0.4486 - val_auc: 0.7155
Epoch 2/8
679/679 [==============================] - 462s 680ms/step - loss: 0.4458 - auc: 0.7186 - val_loss: 0.4415 - val_auc: 0.7268
Epoch 3/8
679/679 [==============================] - 453s 666ms/step - loss: 0.4411 - auc: 0.7269 - val_loss: 0.4425 - val_auc: 0.7246
Epoch 4/8
679/679 [==============================] - 447s 658ms/step - loss: 0.4397 - auc: 0.7294 - val_loss: 0.4428 - val_auc: 0.7276
Epoch 5/8
679/679 [==============================] - 454s 668ms/step - loss: 0.4393 - auc: 0.7305 - val_loss: 0.4404 - val_auc: 0.7299
Epoch 6/8
679/679 [==============================] - 365s 536ms/step - loss: 0.4378 - auc: 0.7334 - val_loss: 0.4432 - val_auc: 0.7275
Epoch 7/8
166/679 [======>.......................] - ETA: 4:19 - loss: 0.4383 - auc: 0.7328

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



679/679 [==============================] - 321s 472ms/step - loss: 0.4572 - auc_1: 0.6999 - val_loss: 0.4492 - val_auc_1: 0.7158
Epoch 5/8
679/679 [==============================] - 356s 524ms/step - loss: 0.4559 - auc_1: 0.7022 - val_loss: 0.4480 - val_auc_1: 0.7183
Epoch 6/8
679/679 [==============================] - 352s 516ms/step - loss: 0.4549 - auc_1: 0.7042 - val_loss: 0.4480 - val_auc_1: 0.7183
Epoch 7/8
679/679 [==============================] - 243s 357ms/step - loss: 0.4544 - auc_1: 0.7052 - val_loss: 0.4488 - val_auc_1: 0.7164
Epoch 8/8
547/679 [=======================>......] - ETA: 40s - loss: 0.4532 - auc_1: 0.7076

...

679/679 [==============================] - 159s 178ms/step - loss: 0.5047 - auc_3: 0.6593 - val_loss: 0.4641 - val_auc_3: 0.7079
Epoch 2/8
679/679 [==============================] - 100s 147ms/step - loss: 0.4658 - auc_3: 0.6937 - val_loss: 0.4526 - val_auc_3: 0.7151
Epoch 3/8
679/679 [==============================] - 102s 149ms/step - loss: 0.4598 - auc_3: 0.7004 - val_loss: 0.4489 - val_auc_3: 0.7193
Epoch 4/8
592/679 [=========================>....] - ETA: 11s - loss: 0.4568 - auc_3: 0.7048

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



679/679 [==============================] - 179s 264ms/step - loss: 0.4129 - auc_4: 0.7775 - val_loss: 0.4219 - val_auc_4: 0.7667
Epoch 4/8
679/679 [==============================] - 193s 283ms/step - loss: 0.4107 - auc_4: 0.7841 - val_loss: 0.4262 - val_auc_4: 0.7669
Epoch 5/8
679/679 [==============================] - 194s 286ms/step - loss: 0.4088 - auc_4: 0.7906 - val_loss: 0.4291 - val_auc_4: 0.7636
Epoch 6/8
679/679 [==============================] - 191s 280ms/step - loss: 0.4068 - auc_4: 0.7975 - val_loss: 0.4357 - val_auc_4: 0.7625
Epoch 7/8
657/679 [============================>.] - ETA: 5s - loss: 0.4044 - auc_4: 0.8043

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



679/679 [==============================] - 78s 115ms/step - loss: 0.4274 - auc_5: 0.7658 - val_loss: 0.4294 - val_auc_5: 0.7624
Epoch 8/8
679/679 [==============================] - 76s 112ms/step - loss: 0.4266 - auc_5: 0.7679 - val_loss: 0.4301 - val_auc_5: 0.7619


## Evaluation

In [12]:
%%time
from collections import defaultdict
from utils import evaluate_model, wAUC

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    for key, eval_model in EVAL_MODELS.items():
        aucs[task_offer_feature][key] = \
                evaluate_model(eval_model, task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)

/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the 

CPU times: user 4h 4min 14s, sys: 19min 39s, total: 4h 23min 54s
Wall time: 1h 2min 52s


In [13]:
from utils import save_metrics
save_metrics(aucs, DATASET, 'ablation')

In [14]:
import pandas as pd
results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name])
        group_by, mask_net, bi_linear_interaction = model_name
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name,
                                        'group_by': group_by, 'mask_net': mask_net,
                                        'bi_linear_interaction': bi_linear_interaction}).to_frame().T],
                            ignore_index=True)

In [15]:
pd.pivot_table(results, 'wAUC', ['group_by', 'mask_net', 'bi_linear_interaction'], 'offers')\
    .rename(columns={'priceCluster': 'price'})[['product_id', 'category3', 'brand', 'category2', 'category1', 'price']]\
    .style.background_gradient(cmap='coolwarm').format(precision=3)

In [16]:
from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=5040, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=5040, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

TypeError: in user code:


    TypeError: tf___keep_nb_events_by_user_by_date() got multiple values for argument 'date_column'


In [ ]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

In [ ]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [ ]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]